# Assignment 04 - Naive Bayes Classifier

## Part 1: Naive Bayes Classifier from scratch

In [1]:
import glob
import pandas as pd
import csv
import re
import string 
from string import digits
import numpy as np

### Loading Data 

#### Loading test data

In [2]:
# positive sentinments in train 
sentiments_train=[]
for name in glob.glob('../IMDB_Dataset/Dataset/train/pos/*'):
    with open(name, encoding="utf8") as fn:
        content = fn.readlines()
        sentiments_train.append([int(name.split("\\")[1].split("_")[0]),content[0],int(name.split("\\")[1].split("_")[1].split(".")[0]),1])

# negative sentiments in train
for name in glob.glob('../IMDB_Dataset/Dataset/train/neg//*'):
    with open(name, encoding="utf8") as fn:
        content = fn.readlines()
        sentiments_train.append([int(name.split("\\")[1].split("_")[0]),content[0],int(name.split("\\")[1].split("_")[1].split(".")[0]),0])
    
# Create the pandas DataFrame
df_train = pd.DataFrame(sentiments_train, columns = ['Id', 'Reviews','Ratings','Sentiments'])
 
# print dataframe.
df_train
    

,Id,Reviews,Ratings,Sentiments
0,0,Bromwell High is a cartoon comedy. It ran at t...,9,1
1,10000,Homelessness (or Houselessness as George Carli...,8,1
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,10,1
3,10002,This is easily the most underrated film inn th...,7,1
4,10003,This is not the typical Mel Brooks film. It wa...,8,1
...,...,...,...,...
24995,9998,"Towards the end of the movie, I felt it was to...",4,0
24996,9999,This is the kind of movie that my enemies cont...,3,0
24997,999,I saw 'Descent' last night at the Stockholm Fi...,3,0
24998,99,Some films that you pick up for a pound turn o...,1,0


#### Loading test data

In [3]:
 # positive sentinments in train 
sentiments_test=[]
for name in glob.glob('../IMDB_Dataset/Dataset/test/pos/*'):
    with open(name, encoding="utf8") as fn:
        content = fn.readlines()
        sentiments_test.append([int(name.split("\\")[1].split("_")[0]),content[0],int(name.split("\\")[1].split("_")[1].split(".")[0]),1])

# negative sentiments in train
for name in glob.glob('../IMDB_Dataset/Dataset/test/neg//*'):
    with open(name, encoding="utf8") as fn:
        content = fn.readlines()
        sentiments_test.append([int(name.split("\\")[1].split("_")[0]),content[0],int(name.split("\\")[1].split("_")[1].split(".")[0]),0])
       
    
# Create the pandas DataFrame
df_test= pd.DataFrame(sentiments_test, columns = ['Id', 'Reviews','Ratings','Sentiments'])
 
# print dataframe.
df_test


,Id,Reviews,Ratings,Sentiments
0,0,I went and saw this movie last night after bei...,10,1
1,10000,Actor turned director Bill Paxton follows up h...,7,1
2,10001,As a recreational golfer with some knowledge o...,9,1
3,10002,"I saw this film in a sneak preview, and it is ...",8,1
4,10003,Bill Paxton has taken the true story of the 19...,8,1
...,...,...,...,...
24995,9998,I occasionally let my kids watch this garbage ...,1,0
24996,9999,When all we have anymore is pretty much realit...,1,0
24997,999,The basic genre is a thriller intercut with an...,3,0
24998,99,Four things intrigued me as to this film - fir...,3,0


### Pre-processing 

#### Removing stop words and cleaning data 

In [6]:
# reading stop_words into file 
f = open("../IMDB_Dataset/Dataset/stop_words.txt", "r")
stop_words=f.read().splitlines()

def cleaning(review):
    review = review.replace("<br>", "").replace("</br>", "")
    review=review.translate(str.maketrans('', '', string.punctuation))
    tokenized_review = review.split(' ')
    review_wsw = [word for word in tokenized_review if not word in stop_words]
    review = (" ").join(review_wsw)
    review=re.sub('[^A-Za-z0-9 ]+', '', review)
    
    
    remove_digits = str.maketrans('', '', digits)
    review= review.translate(remove_digits)
    review = review.rstrip()
    return review.lower()
    
    
    
df_train['Reviews'] = df_train['Reviews'].map(lambda review: cleaning(review)  )
df_test['Reviews'] = df_test['Reviews'].map(lambda review: cleaning(review)  )



df_test


,Id,Reviews,Ratings,Sentiments
0,0,went saw movie last night coaxed friends mine ...,10,1
1,10000,actor turned director bill paxton follows prom...,7,1
2,10001,recreational golfer knowledge sports history p...,9,1
3,10002,saw film sneak preview delightful cinematograp...,8,1
4,10003,bill paxton taken true story us golf open mad...,8,1
...,...,...,...,...
24995,9998,occasionally let kids watch garbage understand...,1,0
24996,9999,anymore pretty much reality tv shows people ma...,1,0
24997,999,basic genre thriller intercut uncomfortable me...,3,0
24998,99,four things intrigued film firstly stars carl...,3,0


### Train Naive Bayes

In [138]:
def trainNaiveBayes(data,classes):
    bigdoc={}
    logprior={}
    N_doc=data.shape[0]
    c=0
    vocabulary=set()
    totalnegv=[]
    totalposv=[]
    classCount=[]
    negvocab=[]
    posvocab=[]
    p_dict={}
    n_dict={}
    
    #creating functions to count the words in the positive and negative sentiments with all reps
    def poscount(sentence):
        [posvocab.append(word) for word in sentence if word!='']
        
    def negcount(sentence):
        [negvocab.append(word) for word in sentence if word!='']
    
    # returnning correct count from either class
    def count(word,class_word):
        count=0
        if class_word==1:
            if word not in p_dict.keys():
                count=0
            else:
                count=p_dict[word]
        if class_word==0:
            if word not in n_dict.keys():
                count=0
            else:
                count=n_dict[word]
            
        return count
    
#   splitting the sentiment column to good and bad and calling function on each value
    data.loc[data['Sentiments'] == 1]['Reviews'].map(lambda review: poscount(review.split(" ")) )
    data.loc[data['Sentiments'] == 0]['Reviews'].map(lambda review:negcount(review.split(" ")) )


#   allocating words from the list of all words with their counts 
    for word in negvocab:
        if word not in n_dict:
            n_dict[word] = 0 
        n_dict[word] += 1
        
    for word in posvocab:
        if word not in p_dict:
            p_dict[word] = 0 
        p_dict[word] += 1
    totalCount=[len(p_dict),len( n_dict)]

    
    
#   getting the entire vocabulary
    data['Reviews'].map(lambda review: [vocabulary.add(word) for word in review.split(" ") if word!=''])

    
    
    for class_data in classes:    
        
        bigdoc[class_data] = data.loc[data['Sentiments'] == class_data]
        N_c=len(bigdoc[class_data])
        logprior[class_data]=np.log(N_c/ N_doc)
        loglikelihood[class_data] = {}

        for word in vocabulary:
            c=count(word,class_data)
            loglikelihood[class_data][word]=np.log( (c+1)/(len(vocabulary)+totalCount[class_data] ))
        
    
    return logprior, loglikelihood, vocabulary

#### Calling the function and getting parameters for train 

In [139]:

data=df_train.copy()
classes=data[data.iloc[:,-1:].columns[0]].unique()

logprior, loglikelihood, vocabulary=trainNaiveBayes(data,classes)


### Test Naive Bayes

In [140]:
def testNaiveBayes(testdoc, logprior, loglikelihood, C, V):
    sum_class = {}
    testdoc= testdoc.split(" ")
    for class_word in classes:
        sum_class[class_word] = logprior[class_word]
        for word in testdoc:
            if word in V:
                sum_class[class_word] = sum_class[class_word] + loglikelihood[class_word][word]
                
    return max(sum_class, key=sum_class.get)

#### Evaluation for the obtained labels

In [141]:
tp = 0
tn = 0
fp = 0
fn = 0
groundLabel=df_test['Sentiments'].to_list()
predictedlabels=[]

df_test['Reviews'].map(lambda review: predictedlabels.append(testNaiveBayes(review, logprior, loglikelihood, [0, 1], vocabulary))  )

for i in range(df_test.shape[0]):
    if (groundLabel[i]== 1 and predictedlabels[i]==1 ):
        tp = tp + 1
    elif (groundLabel[i]== 0 and predictedlabels[i]==0 ):
        tn = tn + 1
    elif (groundLabel[i]== 0 and predictedlabels[i]==1 ):
        fp = fp + 1
    elif (groundLabel[i]== 1 and predictedlabels[i]==0 ):
        fn = fn + 1



### Confusion matrix and Accuracy

In [146]:
confusion_matrix = np.array([ [tp, fp], [fn, tn] ])
accuracy = (tn + tp) / (tn + tp + fn + fp)
display("Confusion Matrix: ",confusion_matrix)
print("Accuracy:", accuracy*100,"%")

'Confusion Matrix: '

array([[10700,  2418],
       [ 1800, 10082]])

Accuracy: 83.128 %


## Part 2: Naive Bayes scikit-learn

#### Importing libraries 

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

#### Vectorizing our data

In [145]:
countVector = CountVectorizer(stop_words= stop_words)

#### Fit and transform data

In [149]:
df_train_matrix = countVector.fit_transform(df_train['Reviews'].values.flatten().tolist())
df_test_matrix = countVector.transform(df_test['Reviews'].values.flatten().tolist())

#### Initiating Multi-Nomial Classifier

In [150]:
nb = MultinomialNB()
nb.fit(df_train_matrix, df_train['Sentiments'].values.flatten().tolist())
PredictedVals = nb.predict(df_test_matrix)

#### Stating Evaluation report

In [153]:

print("Confusion matrix: \n",metrics.confusion_matrix(df_test['Sentiments'].to_list(), PredictedVals))
print("Accuracy: ", metrics.accuracy_score(df_train['Sentiments'].to_list(), PredictedVals) * 100, "%")

Confusion matrix: 
 [[10993  1507]
 [ 2884  9616]]
Accuracy:  82.43599999999999 %
